
<font size="20"><h3 align='center'>**_DSL Project_**</font><br>
    
    
<font size="5"><h3 align='center'><span style='color:Blue'> Predict impact of air quality on mortality rates(Kaggle Competition)</font>


<font size="3"><h3 align='center'>https://www.kaggle.com/c/predict-impact-of-air-quality-on-death-rates/overview</font><br>
    <br>


<font size="3">**Group Number - 2**<br>
<font size="3"> Mimansi Agarwal   &emsp;19BM6JP15</font>        
<font size="3"> Vaibhav HK Anand  &emsp;19BM6JP24</font>    
<font size="3"> Bankar Prasad Vilas &emsp;19BM6JP25</font> 


# Library

In [1]:

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from impyute.imputation.cs import mice
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import  make_scorer
import math

# Data import

In [2]:
train = pd.read_csv('C:/Users/deciPh3r/Desktop/Project/files/train.csv', parse_dates=['date'], index_col='Id')
test = pd.read_csv('C:/Users/deciPh3r/Desktop/Project/files/test.csv', parse_dates=['date'], index_col='Id')
region=pd.read_csv('C:/Users/deciPh3r/Desktop/Project/files/regions.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18403 entries, 1 to 18403
Data columns (total 8 columns):
region            18403 non-null object
date              18403 non-null datetime64[ns]
mortality_rate    18403 non-null float64
O3                18394 non-null float64
PM10              18394 non-null float64
PM25              15127 non-null float64
NO2               11833 non-null float64
T2M               18403 non-null float64
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 1.3+ MB


**_Missing values present in 4 columns, O3 ,PM10, PM25, NO2_**

In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7886 entries, 18404 to 26289
Data columns (total 7 columns):
region    7886 non-null object
date      7886 non-null datetime64[ns]
O3        7886 non-null float64
PM10      7886 non-null float64
PM25      7886 non-null float64
NO2       7886 non-null float64
T2M       7886 non-null float64
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 492.9+ KB


In [5]:
region

,Code,Region
0,E12000001,North East
1,E12000002,North West
2,E12000003,Yorkshire and The Humber
3,E12000004,East Midlands
4,E12000005,West Midlands
5,E12000006,East of England
6,E12000007,London
7,E12000008,South East
8,E12000009,South West


In [6]:
region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
Code      9 non-null object
Region    9 non-null object
dtypes: object(2)
memory usage: 272.0+ bytes


# Exploratory data analysis

In [7]:
region.head()

,Code,Region
0,E12000001,North East
1,E12000002,North West
2,E12000003,Yorkshire and The Humber
3,E12000004,East Midlands
4,E12000005,West Midlands


#  Imputation

## Impution on full data

#  Feature Engeering

one hot encoding for regions

In [14]:
dummy1= pd.get_dummies(train['region'])
train_f = train.join(dummy1)

dummy2= pd.get_dummies(test['region'])
test_f = test.join(dummy2)

In [15]:
train_f.shape,test_f.shape

((18403, 17), (7886, 16))

doy and year

In [16]:
train_f['year'] = train_f['date'].map(lambda x: x.year)
test_f['year'] = test_f['date'].map(lambda x: x.year)

train_f['year^2'] = train_f['year']**2
test_f['year^2'] = test_f['year']**2

train_f['doy'] = train_f['date'].map(lambda x: x.dayofyear)
test_f['doy'] = test_f['date'].map(lambda x: x.dayofyear)

train_f['doy^2'] = train_f['doy']**2
test_f['doy^2'] = test_f['doy']**2

Log Transformation

In [19]:
train_f['NO2_log']=np.log(train_f['NO2'])
train_f['PM10_log']=np.log(train_f['PM10'])
train_f['PM25_log']=np.log(train_f['PM25'])

test_f['NO2_log']=np.log(test_f['NO2'])
test_f['PM10_log']=np.log(test_f['PM10'])
test_f['PM25_log']=np.log(test_f['PM25'])

In [20]:
train_f.info(),test_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18403 entries, 1 to 18403
Data columns (total 24 columns):
region            18403 non-null object
date              18403 non-null datetime64[ns]
mortality_rate    18403 non-null float64
O3                18394 non-null float64
PM10              18394 non-null float64
PM25              15127 non-null float64
NO2               11833 non-null float64
T2M               18403 non-null float64
E12000001         18403 non-null uint8
E12000002         18403 non-null uint8
E12000003         18403 non-null uint8
E12000004         18403 non-null uint8
E12000005         18403 non-null uint8
E12000006         18403 non-null uint8
E12000007         18403 non-null uint8
E12000008         18403 non-null uint8
E12000009         18403 non-null uint8
year              18403 non-null int64
year^2            18403 non-null int64
doy               18403 non-null int64
doy^2             18403 non-null int64
NO2_log           11833 non-null float64
PM10_log 

(None, None)

In [22]:
total_f=total=pd.concat([train_f,test_f],sort=False)

In [23]:
total_f['T2M_rolling'] = total_f.groupby('region')['T2M'].transform(lambda x: x.rolling(30).mean())

In [24]:
total_f.shape

(26289, 25)

In [29]:
train_f=total_f[:18403].dropna()
test_f=total_f[18403:].drop(['mortality_rate'],axis=1)

In [30]:
train_f.shape,test_f.shape

((11833, 25), (7886, 24))

#  Modelling - Regression based models

In [32]:
X_train, X_test, y_train, y_test = train_test_split(train_f.drop(['mortality_rate'],axis=1), train_f['mortality_rate'], test_size=0.2)

In [34]:
X_train.columns

Index(['region', 'date', 'O3', 'PM10', 'PM25', 'NO2', 'T2M', 'E12000001',
       'E12000002', 'E12000003', 'E12000004', 'E12000005', 'E12000006',
       'E12000007', 'E12000008', 'E12000009', 'year', 'year^2', 'doy', 'doy^2',
       'NO2_log', 'PM10_log', 'PM25_log', 'T2M_rolling'],
      dtype='object')

In [87]:
RMSE= make_scorer(metrics.mean_squared_error)

In [88]:
predictors=['O3','PM10','PM25','NO2','T2M']

In [89]:
X=train_f.drop(['mortality_rate'],axis=1)
y=train_f['mortality_rate']

In [97]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"MSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression MSE [0.25590564 0.2384088  0.23534413 0.24975171 0.21540355] 

average RMSE : 0.2389627659994164 
 

Lasso MSE [0.26501773 0.24556641 0.2426855  0.25132788 0.21040855] 

average RMSE : 0.243001215350219 
 

Ridge MSE [0.25590577 0.23840889 0.23534414 0.24975171 0.21540335] 

average RMSE : 0.2389627739056404 
 

RandomForestRegressor MSE [0.27209648 0.25170445 0.26211702 0.26120663 0.21941687] 

average RMSE : 0.2533082876488943 
 



In [38]:
predictors=['O3','PM10','PM25','NO2_log','T2M']

In [39]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression MSE [0.07037094 0.05958156 0.06063343 0.06502241 0.04669236] 

average RMSE : 0.24588643224768653 
 

Lasso MSE [0.0777591  0.06480518 0.06700435 0.0706307  0.04414869] 

average RMSE : 0.25469511474052214 
 

Ridge MSE [0.0703723  0.05958211 0.06063517 0.06502132 0.04668927] 

average RMSE : 0.24588621931721555 
 

RandomForestRegressor MSE [0.07403686 0.06335579 0.06870533 0.0682289  0.0481437 ] 

average RMSE : 0.2539569197562435 
 



for NO2 log transformation   ::


imporved rmse for each model, but for RFR it didn't change much. But there's huge improvement in LASSO

In [40]:
predictors=['O3','PM10','PM25_log','NO2_log','T2M']

In [41]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression MSE [0.07061605 0.05946335 0.06050952 0.06545444 0.04673174] 

average RMSE : 0.24607929831932834 
 

Lasso MSE [0.0777591  0.06480518 0.06701569 0.0706307  0.04416158] 

average RMSE : 0.2547046263577067 
 

Ridge MSE [0.07061648 0.05946357 0.06051137 0.06545193 0.04672744] 

average RMSE : 0.2460775452509909 
 

RandomForestRegressor MSE [0.07403686 0.06335579 0.06870533 0.0682289  0.0481437 ] 

average RMSE : 0.2539569197562435 
 



In [83]:
predictors=['O3','PM10_log','PM25_log','NO2_log','T2M']

In [84]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression MSE [0.0704298  0.0594     0.06053525 0.06535633 0.0470255 ] 

average RMSE : 0.24606782884457915 
 

Lasso MSE [0.07800644 0.06420953 0.06746213 0.0708699  0.04398151] 

average RMSE : 0.25476636609194875 
 

Ridge MSE [0.07043066 0.05940042 0.06053717 0.06535372 0.04701942] 

average RMSE : 0.24606559284124743 
 

RandomForestRegressor MSE [0.07403686 0.06335579 0.06870533 0.0682289  0.0481437 ] 

average RMSE : 0.2539569197562435 
 



In [44]:
predictors=['O3','PM10_log','PM25_log','NO2_log','T2M_rolling']

In [45]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression MSE [0.0632786  0.05158769 0.05454941 0.05465633 0.03754879] 

average RMSE : 0.2287447625578133 
 

Lasso MSE [0.07552484 0.05814664 0.0628718  0.06421509 0.03673123] 

average RMSE : 0.24392195451763216 
 

Ridge MSE [0.06328049 0.05158629 0.05455079 0.05465364 0.03754439] 

average RMSE : 0.22874247605793074 
 

RandomForestRegressor MSE [0.07424195 0.06026976 0.06748551 0.06670099 0.0377598 ] 

average RMSE : 0.24757140263315927 
 



In [46]:
predictors=['O3','PM10_log','PM25_log','NO2_log','T2M_rolling','year','year^2','doy','doy^2']

In [47]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression MSE [0.05407892 0.05722477 0.05167371 0.05136022 0.12879596] 

average RMSE : 0.2619670084440214 
 

Lasso MSE [0.06533548 0.05770386 0.05879518 0.07073451 0.05900798] 

average RMSE : 0.2496305286183826 
 

Ridge MSE [0.0550094  0.05066966 0.05292685 0.0529487  0.05559023] 

average RMSE : 0.23114706953558675 
 

RandomForestRegressor MSE [0.07179963 0.05789396 0.0650419  0.06865549 0.03689816] 

average RMSE : 0.24506699186563816 
 



In [57]:
predictors=['T2M_rolling', 'year', 'year^2', 'doy', 'doy^2','E12000001', 'E12000002', 'E12000003', 'E12000004','E12000005', 'E12000006', 'E12000007', 'E12000008']

In [58]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression MSE [0.04168    0.03406252 0.03068743 0.02720154 0.06160898] 

average RMSE : 0.19760590803077888 
 

Lasso MSE [0.06552461 0.05884921 0.06196126 0.07262577 0.05672193] 

average RMSE : 0.25126988704022435 
 

Ridge MSE [0.0407161  0.03360847 0.03018191 0.02743355 0.03826432] 

average RMSE : 0.18450167805835072 
 

RandomForestRegressor MSE [0.06121355 0.0508713  0.03984644 0.04632081 0.03840608] 

average RMSE : 0.21755834997017573 
 



In [53]:
predictors1=['NO2_log', 'year', 'year^2', 'doy', 'doy^2','PM10_log']
predictors2=['T2M_rolling', 'year', 'year^2', 'doy', 'doy^2','E12000001', 'E12000002', 'E12000003', 'E12000004','E12000005', 'E12000006', 'E12000007', 'E12000008']
predictors3=['E12000001', 'E12000002', 'E12000003', 'E12000004','E12000005', 'E12000006', 'E12000007', 'E12000008']

In [98]:
pred = [
    predictors1,
    predictors2,
    predictors3,
]
for preds in pred:
    #model_name = model.__class__.__name__
    
    accuracies = cross_val_score(LinearRegression(),X[preds], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

RandomForestRegressor RMSE [0.23201375 0.26048128 0.22666889 0.25644528 0.42542438] 

average RMSE : 0.28020671583860196 
 

RandomForestRegressor RMSE [0.20415681 0.18456036 0.17517828 0.16492889 0.24821156] 

average RMSE : 0.19540717885674203 
 

RandomForestRegressor RMSE [0.28887085 0.26111306 0.24310381 0.22983521 0.22967401] 

average RMSE : 0.25051938878501434 
 



## Final Algo - mixed LR

In [75]:
a = LinearRegression()
b = LinearRegression()
c = LinearRegression()
ab = LinearRegression()

In [78]:

y0 = train_f['mortality_rate']



#modela
a_predictors = ['NO2_log', 'year', 'year^2', 'doy', 'doy^2','PM10_log']
a_X0 = train_f[a_predictors]
a.fit(a_X0, y0)



#modelb
b_predictors = [
        'T2M_rolling', 'year', 'year^2', 'doy', 'doy^2',
        'E12000001', 'E12000002', 'E12000003', 'E12000004',
        'E12000005', 'E12000006', 'E12000007', 'E12000008'
        ]
b_X0 = train_f[b_predictors]
b.fit(b_X0, y0)



#modelc
c_predictors = [
        'E12000001', 'E12000002', 'E12000003', 'E12000004',
        'E12000005', 'E12000006', 'E12000007', 'E12000008'
        ]
c_X0 = train_f[c_predictors]
c.fit(c_X0, y0)


region_effect_reduction = 0.85


# Model AB
# Produces the final solution. Combines A and "reduced B" by using their predictions as features for a linear regression.

ab_X0 = pd.DataFrame({
    'A': a.predict(a_X0),
    'reduced B': b.predict(b_X0) - (c.predict(c_X0) * region_effect_reduction)
    })

ab.fit(ab_X0, y0)

# predict

a_X1 = test_f[a_predictors]
b_X1 = test_f[b_predictors]
c_X1 = test_f[c_predictors]

ab_X1 = pd.DataFrame({'A': a.predict(a_X1),'reduced B': b.predict(b_X1) - (c.predict(c_X1) * region_effect_reduction)})

ab_y1 = ab.predict(ab_X1)



predictions = pd.DataFrame(index=test_f.index)
predictions['mortality_rate'] =  a.predict(a_X1)
predictions.to_csv('C:/Users/deciPh3r/Desktop/Project/files/sub/final_0.0.csv')


# Results
best LB socre for -(region_effect_reduction = 0.85)<br>
private score - 0.28948<br>
public score - 0.28784<br>


benchmark -<br>
private score - 0.29022<br>
public socre - 0.29259<br>